In [5]:
import os
import requests
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# MongoDB Atlas connection setup
def get_mongo_client():
    mongo_uri = os.getenv("MONGO_URI")  # Get the MongoDB URI from .env file
    if not mongo_uri:
        raise ValueError("MONGO_URI is not set in the .env file")
    client = MongoClient(mongo_uri)
    return client

# Download CSV if not exists
def download_csv():
    if not os.path.exists("./FinalPermutations.csv"):
        s3_url = "https://coolmeal.s3.amazonaws.com/FinalPermutations.csv"
        print(f"File not found at ./FinalPermutations.csv. Downloading from S3...")
        response = requests.get(s3_url)

        if response.status_code == 200:
            with open("./FinalPermutations.csv", "wb") as f:
                f.write(response.content)
            print(
                f"File downloaded successfully from S3 and saved to ./FinalPermutations.csv."
            )
        else:
            raise FileNotFoundError(
                f"Failed to download file from {s3_url}. HTTP status code: {response.status_code}"
            )

# Load data from CSV to MongoDB
def load_data_to_mongo():
    # Download the CSV file if not present
    download_csv()

    # Read the CSV data
    print("Reading data from ./FinalPermutations.csv...")
    df = pd.read_csv("./FinalPermutations.csv")
    print(f"Data loaded successfully. Number of records: {len(df)}")

    # Get the first 10 records
    first_10_records = df.head(10)

    # Add index as a separate column (e.g., 'index')
    first_10_records_with_index = first_10_records.copy()
    first_10_records_with_index['index'] = first_10_records_with_index.index

    # Convert DataFrame to a list of dictionaries
    records_to_insert = first_10_records_with_index.to_dict(orient='records')

    print(f"Uploading the first 10 records with index: {records_to_insert}")

    client = get_mongo_client()
    db = client["cool-server"]  
    collection = db["Meal_plans"]  

    print("Connected to MongoDB Atlas successfully.")

    # Insert the records into MongoDB
    print("Inserting data into MongoDB Atlas...")
    collection.insert_many(records_to_insert)
    print("First 10 records with index inserted into MongoDB successfully.")

# Run the function to load data
load_data_to_mongo()


Reading data from ./FinalPermutations.csv...
Data loaded successfully. Number of records: 200000
Uploading the first 10 records with index: [{'Breakfast': 'Red Rice with Fish Curry, Tapioca Chips, Batu Moju', 'Lunch': 'Red Rice with Banana Blossom Curry, Green Gram Curry, Fenugreek Curry', 'Dinner': 'Vegetable Kottu with Devilled Fish', 'Price': 792.3, 'Total Energy(Kcal)': 1482.37, 'Total Protein(g)': 73.5357, 'Total Total fat(g)': 83.1566, 'Total Carbohydrates(g)': 175.93710000000002, 'Total Magnesium(mg)': 478.5, 'Total Sodium(mg)': 128.367, 'Total Potassium(mg)': 3346.98, 'Total Saturated Fatty Acids(mg)': 37105.9815, 'Total Monounsaturated Fatty Acids(mg)': 6314.2741000000005, 'Total Polyunsaturated Fatty Acids(mg)': 5152.7615000000005, 'Total Free sugar(g)': 11.7442, 'Total Starch(g)': 151.1037, 'index': 0}, {'Breakfast': 'Red Rice with Parippu, Brinjal Moju, Ginger Chicken', 'Lunch': 'Kottu Roti with Chicken Curry', 'Dinner': 'Red Rice with Fish Curry, Kiri Hodi, Sprats Fry', 'P